# Trabajo Práctico 2: Reseñas de Películas - Naive Bayes

## Grupo 11 - "Los Outliers"

- Castillo, Carlos
- Destefanis, Juan Pablo
- Gómez, Celeste

# Dependencias

In [1]:
!pip install nltk
!python -m spacy download es_core_news_sm
!python -m spacy download es

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 57.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.7.0/es_core_news_sm-3.7.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 

In [2]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import spacy

from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
from sklearn.naive_bayes import MultinomialNB

import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline

from joblib import dump, load
from os.path import exists

In [4]:
from unidecode import unidecode

# Carga de datos

Cargo el dataset de entrenamiento

In [5]:
df_train = pd.read_csv("https://drive.usercontent.google.com/u/1/uc?id=17JRXErnbhnHxhD8Z8AFfOP4fJnkivu0-&export=download", index_col=0)

Cargo el dataset de testing

In [6]:
df_test = pd.read_csv("https://drive.usercontent.google.com/u/1/uc?id=1HtgxC7iumrAXTsaAS6Qkp0w25eerGc4w&export=download", index_col=0)

# Naive Bayes con Bag of Words simple

Creo el vectorizer para armar la bag of words

In [7]:
vectorizer = CountVectorizer()

Creo una primera funcion de limpieza para poner el texto en minúsculas y eliminar diacríticos

In [8]:
def limpiar_texto(texto):
    #Convertir texto a minúsculas
    texto = texto.lower()

    #Eliminar diacríticos usando la biblioteca unidecode
    texto = unidecode(texto)

    return texto

Copio el dataset original

In [9]:
df_t_c1 = df_train.copy()

Limpio los textos y veo como quedaron algunos

In [10]:
df_t_c1["rev_limpia"] = df_t_c1["review_es"].apply(limpiar_texto)

In [11]:
df_t_c1["rev_limpia"]

ID
0        uno de los otros criticos ha mencionado que de...
1        una pequena pequena produccion.la tecnica de f...
2        pense que esta era una manera maravillosa de p...
3        basicamente, hay una familia donde un nino peq...
4        el "amor en el tiempo" de petter mattei es una...
                               ...                        
49995    pense que esta pelicula hizo un buen trabajo a...
49996    mala parcela, mal dialogo, mala actuacion, dir...
49997    soy catolica ensenada en escuelas primarias pa...
49998    voy a tener que estar en desacuerdo con el com...
49999    nadie espera que las peliculas de star trek se...
Name: rev_limpia, Length: 50000, dtype: object

## Generación de train y test para cross validation

Genero los conjuntos de train y test para cross validation (sobre el conjunto de train)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_t_c1.review_es, df_t_c1.sentimiento, test_size=0.4, random_state=13, stratify=df_t_c1.sentimiento)

## Entrenamiento

Creo la bag of words entrenando sobre el vectorizer sobre el conjunto de train.

In [13]:
count_vect = CountVectorizer(preprocessor=limpiar_texto,
                             min_df=5
                             )

X_train = count_vect.fit_transform(X_train)
X_test = count_vect.transform(X_test)

<30000x39157 sparse matrix of type '<class 'numpy.int64'>'
	with 3929118 stored elements in Compressed Sparse Row format>

Entreno Naive Bayes directamente en cross validation.

In [14]:
clf = MultinomialNB(alpha=1)

kfoldcv = StratifiedKFold()

scores_cv_nb_base = cross_val_score(clf,
                                    X_train, y_train,
                                    cv = 10,
                                    scoring='f1_macro',
                                    )

scores_cv_nb_base

array([0.83887667, 0.83980748, 0.84449388, 0.83614762, 0.84278477,
       0.84548453, 0.83343034, 0.83987446, 0.82272106, 0.82676284])

In [15]:
clf.fit(X_train, y_train)

MultinomialNB(alpha=1)

Predigo los valores de test.

In [16]:
y_nb_base = clf.predict(X_test)

In [17]:
y_nb_base

array(['positivo', 'positivo', 'positivo', ..., 'negativo', 'positivo',
       'positivo'], dtype='<U8')

Visualizo la matriz de confusion

In [18]:
print("Matriz de confusión\n",confusion_matrix(y_test, y_nb_base))

Matriz de confusión
 [[8657 1343]
 [1952 8048]]


## Predicciones

Aplico la funcion de preprocesamiento al conjunto de test y utilizo el vectorizer entrenado con el conjunto de train para transformar los datos de test

In [19]:
df_test["reviews"] = df_test["review_es"].apply(limpiar_texto)

In [20]:
X_test_test = count_vect.transform(df_test.reviews)

Predigo con el modelo entrenado

In [21]:
y_pred_test_nb_base = clf.predict(X_test_test)

Creo un dataframe para exportar las predicciones a un .csv

In [22]:
df_export = df_test.copy()

Agrego las predicciones al dataframe

In [23]:
df_export["sentimiento"] = y_pred_test_nb_base

Quito las columnas de reviews.

In [24]:
df_export.drop(["reviews"], axis="columns", inplace=True)

In [25]:
df_export.drop(["review_es"], axis="columns", inplace=True)

Exporto las predicciones.

In [26]:
df_export.to_csv("nb_base_sin_preproc.csv")

In [27]:
dump(clf, "naive_bayes_inicial.joblib")

['naive_bayes_inicial.joblib']